# KAN MODEL BASATO SU SATELLITE

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
from torchsummary import summary
import math

# Classe KANLinear (Layer Personalizzato)
class KANLinear(nn.Module):
    def __init__(self, in_features, out_features, grid_size=5, spline_order=3, 
                 scale_noise=0.1, scale_base=1.0, scale_spline=1.0, 
                 enable_standalone_scale_spline=True, base_activation=nn.SiLU, 
                 grid_eps=0.02, grid_range=[-1, 1]):
        super(KANLinear, self).__init__()
        # Inizializzazione come nel tuo codice originale
        self.in_features = in_features
        self.out_features = out_features
        self.grid_size = grid_size
        self.spline_order = spline_order

        h = (grid_range[1] - grid_range[0]) / grid_size
        grid = ((torch.arange(-spline_order, grid_size + spline_order + 1) * h 
                 + grid_range[0]).expand(in_features, -1).contiguous())
        self.register_buffer("grid", grid)

        self.base_weight = nn.Parameter(torch.Tensor(out_features, in_features))
        self.spline_weight = nn.Parameter(
            torch.Tensor(out_features, in_features, grid_size + spline_order)
        )
        if enable_standalone_scale_spline:
            self.spline_scaler = nn.Parameter(torch.Tensor(out_features, in_features))

        self.scale_noise = scale_noise
        self.scale_base = scale_base
        self.scale_spline = scale_spline
        self.enable_standalone_scale_spline = enable_standalone_scale_spline
        self.base_activation = base_activation()
        self.grid_eps = grid_eps

        self.reset_parameters()

    def reset_parameters(self):
        nn.init.kaiming_uniform_(self.base_weight, a=math.sqrt(5) * self.scale_base)
        with torch.no_grad():
            noise = ((torch.rand(self.grid_size + 1, self.in_features, self.out_features) - 0.5)
                     * self.scale_noise / self.grid_size)
            self.spline_weight.data.copy_(
                self.scale_spline * self.curve2coeff(self.grid.T[self.spline_order : -self.spline_order], noise)
            )
            if self.enable_standalone_scale_spline:
                nn.init.kaiming_uniform_(self.spline_scaler, a=math.sqrt(5) * self.scale_spline)

    def b_splines(self, x):
        # Calcolo delle basi spline
        grid = self.grid
        x = x.unsqueeze(-1)
        bases = ((x >= grid[:, :-1]) & (x < grid[:, 1:])).to(x.dtype)
        for k in range(1, self.spline_order + 1):
            bases = ((x - grid[:, :-(k+1)]) / (grid[:, k:-1] - grid[:, :-(k+1)]) * bases[:, :, :-1]
                     + (grid[:, k+1:] - x) / (grid[:, k+1:] - grid[:, 1:-k]) * bases[:, :, 1:])
        return bases.contiguous()

    def curve2coeff(self, x, y):
        # Calcolo dei coefficienti delle spline
        A = self.b_splines(x).transpose(0, 1)
        B = y.transpose(0, 1)
        solution = torch.linalg.lstsq(A, B).solution
        return solution.permute(2, 0, 1).contiguous()

    def forward(self, x):
        x = x.view(x.size(0), -1)
        base_output = F.linear(self.base_activation(x), self.base_weight)
        spline_output = F.linear(
            self.b_splines(x).view(x.size(0), -1), 
            self.spline_weight.view(self.out_features, -1)
        )
        return base_output + spline_output

# Modello ConvNeXtKAN
class ConvNeXtKAN(nn.Module):
    def __init__(self):
        super(ConvNeXtKAN, self).__init__()
        # Modello di base
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False),  # Primo layer convoluzionale
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool2d((1, 1))
        )
        self.flatten = nn.Flatten()

        # Layer KAN
        self.kan1 = KANLinear(128, 256)
        self.kan2 = KANLinear(256, 2)  # 2 classi

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.flatten(x)
        x = self.kan1(x)
        x = self.kan2(x)
        return x

# Stampa del modello
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ConvNeXtKAN().to(device)

# Stampa un riassunto del modello per immagini 1x150x150
summary(model, (1, 150, 150))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 75, 75]           3,136
       BatchNorm2d-2           [-1, 64, 75, 75]             128
              ReLU-3           [-1, 64, 75, 75]               0
         MaxPool2d-4           [-1, 64, 38, 38]               0
            Conv2d-5          [-1, 128, 38, 38]          73,728
       BatchNorm2d-6          [-1, 128, 38, 38]             256
              ReLU-7          [-1, 128, 38, 38]               0
 AdaptiveAvgPool2d-8            [-1, 128, 1, 1]               0
           Flatten-9                  [-1, 128]               0
             SiLU-10                  [-1, 128]               0
        KANLinear-11                  [-1, 256]               0
             SiLU-12                  [-1, 256]               0
        KANLinear-13                    [-1, 2]               0
Total params: 77,248
Trainable params: 

In [35]:
import torch
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F

# Assumiamo che i tuoi dati siano già in formato numpy e normalizzati tra 0 e 1
# Carica i dati come tensori
train_images_tensor = torch.tensor(augmented_train_images_normalized, dtype=torch.float32)
val_images_tensor = torch.tensor(val_images, dtype=torch.float32)

# Aggiungi una dimensione per il canale (1 canale per immagini in scala di grigio)
train_images_tensor = train_images_tensor.squeeze(-1)  # Shape: (7750, 1, 150, 150)
val_images_tensor = val_images_tensor.squeeze(-1)      # Shape: (16, 1, 150, 150)

# Aggiungi esplicitamente la dimensione per il canale
train_images_tensor = train_images_tensor.unsqueeze(1)  # Shape: (7750, 1, 150, 150)
val_images_tensor = val_images_tensor.unsqueeze(1)      # Shape: (16, 1, 150, 150)

# Crea i target (labels) come tensori (assumiamo che i target siano già pronti)
train_labels_tensor = torch.tensor(augmented_train_labels, dtype=torch.long)  # Shape: (7750,)
val_labels_tensor = torch.tensor(val_labels, dtype=torch.long)      # Shape: (16,)

# Crea il dataset e DataLoader
train_dataset = TensorDataset(train_images_tensor, train_labels_tensor)
val_dataset = TensorDataset(val_images_tensor, val_labels_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Modello e device (GPU/CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ConvNeXtKAN().to(device)

# Definizione della funzione di perdita e ottimizzatore
criterion = nn.CrossEntropyLoss()  # Per classificazione multi-classe o binaria
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Funzione di allenamento con Early Stopping
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=100, patience=10):
    best_val_accuracy = 0.0
    best_val_loss = float("inf")
    epochs_without_improvement = 0  # Conta quante epoche senza miglioramento

    for epoch in range(num_epochs):
        model.train()  # Modalità allenamento
        running_loss = 0.0
        correct_preds = 0
        total_preds = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            # Azzeramento dei gradienti
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)

            # Calcolo della perdita
            loss = criterion(outputs, labels)

            # Backpropagation
            loss.backward()
            optimizer.step()

            # Calcolo delle metriche
            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct_preds += torch.sum(preds == labels).item()
            total_preds += labels.size(0)

        # Calcolo della loss media e accuratezza
        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_accuracy = correct_preds / total_preds

        print(f"Epoch {epoch + 1}/{num_epochs}")
        print(f"Train Loss: {epoch_loss:.4f}, Train Accuracy: {epoch_accuracy:.4f}")

        # Fase di validazione
        model.eval()  # Modalità valutazione
        val_loss = 0.0
        val_correct_preds = 0
        val_total_preds = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)

                # Forward pass
                outputs = model(inputs)

                # Calcolo della perdita
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)

                # Calcolo delle metriche
                _, preds = torch.max(outputs, 1)
                val_correct_preds += torch.sum(preds == labels).item()
                val_total_preds += labels.size(0)

        # Calcolo della loss e accuratezza di validazione
        val_loss /= len(val_loader.dataset)
        val_accuracy = val_correct_preds / val_total_preds

        print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

        # Early Stopping: fermarsi se non c'è miglioramento
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_val_accuracy = val_accuracy
            epochs_without_improvement = 0
            # Salva il modello con la miglior loss di validazione
            torch.save(model.state_dict(), "best_model.pth")
            print("Model saved!")
        else:
            epochs_without_improvement += 1
            print(f"Early Stopping Counter: {epochs_without_improvement}/{patience}")

        # Se il numero di epoche senza miglioramenti supera la pazienza, fermati
        if epochs_without_improvement >= patience:
            print("Early stopping triggered. Stopping training.")
            break

# Avvia l'allenamento
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=100, patience=10)


Epoch 1/100
Train Loss: 0.3313, Train Accuracy: 0.8541
Validation Loss: 0.6979, Validation Accuracy: 0.6875
Model saved!
Epoch 2/100
Train Loss: 0.2132, Train Accuracy: 0.9125
Validation Loss: 2.6177, Validation Accuracy: 0.6875
Early Stopping Counter: 1/10
Epoch 3/100
Train Loss: 0.1774, Train Accuracy: 0.9262
Validation Loss: 1.7192, Validation Accuracy: 0.6250
Early Stopping Counter: 2/10
Epoch 4/100
Train Loss: 0.1625, Train Accuracy: 0.9342
Validation Loss: 3.6267, Validation Accuracy: 0.6250
Early Stopping Counter: 3/10
Epoch 5/100
Train Loss: 0.1782, Train Accuracy: 0.9324
Validation Loss: 1.0677, Validation Accuracy: 0.6250
Early Stopping Counter: 4/10
Epoch 6/100
Train Loss: 0.1483, Train Accuracy: 0.9400
Validation Loss: 2.8287, Validation Accuracy: 0.5000
Early Stopping Counter: 5/10
Epoch 7/100
Train Loss: 0.1370, Train Accuracy: 0.9506
Validation Loss: 0.9447, Validation Accuracy: 0.6875
Early Stopping Counter: 6/10
Epoch 8/100
Train Loss: 0.1359, Train Accuracy: 0.9472
Va

In [14]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import f1_score
import torch.nn.functional as F

criterion = nn.CrossEntropyLoss()  # Per classificazione multi-classe o binaria

# Carica il modello salvato
model = ConvNeXtKAN().to(device)
model.load_state_dict(torch.load("best_model.pth"))
model.eval()  # Modalità di valutazione

# Dati di test
test_images_tensor = torch.tensor(test_images, dtype=torch.float32)  # Usa i tuoi dati di test
test_labels_tensor = torch.tensor(test_labels, dtype=torch.long)  # Usa le tue etichette di test

# Aggiungi la dimensione del canale per il modello (1 canale per immagini in scala di grigio)
test_images_tensor = test_images_tensor.unsqueeze(1)  # Shape: (n_test, 1, 150, 150)

# Crea il dataset e DataLoader per il test
test_dataset = TensorDataset(test_images_tensor, test_labels_tensor)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Calcola la loss e l'accuracy sui dati di test
test_loss = 0.0
correct_preds = 0
total_preds = 0
all_preds = []
all_labels = []

with torch.no_grad():  # Non calcolare i gradienti durante il test
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Forward pass
        outputs = model(inputs)

        # Calcolo della perdita
        loss = criterion(outputs, labels)
        test_loss += loss.item() * inputs.size(0)

        # Calcolo delle predizioni
        _, preds = torch.max(outputs, 1)
        correct_preds += torch.sum(preds == labels).item()
        total_preds += labels.size(0)

        # Memorizza le predizioni e le etichette per il calcolo di F1
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Calcolo della loss media e accuratezza finale
test_loss /= len(test_loader.dataset)
test_accuracy = correct_preds / total_preds

# Calcola F1 score
f1 = f1_score(all_labels, all_preds, average='weighted')

print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")

/var/folders/tj/7m7f0dn51772hz65_qmb34940000gn/T/ipykernel_54500/3871363675.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_model

Test Loss: 0.6367
Test Accuracy: 0.7692
F1 Score: 0.7722
